## Instructions of use

We need to feed this program with a csv file containing these columns ['Id_Metadatos', 'Instancia', 'BaseDeDatos', 'Esquema', 'TipoDeObjeto',
       'NombreDeObjeto', 'Descripcion', 'Contenido']. This program will automatically split cols by tables and save files by schemas. 

## Data input and preparation

Libraries importation

In [17]:
import re 
import pandas as pd 
import numpy as np 
import os 
import warnings



Detection of encoding system

In [27]:
file_path=r"routeimport chardet
with open(file_path, "rb") as f:
    result = chardet.detect(f.read(10000))  # Read first 10KB
    print(result)
    coder=result['encoding']

{'encoding': 'UTF-8-SIG', 'confidence': 1.0, 'language': ''}


Data reading

In [28]:
data = pd.read_csv(file_path, encoding=coder, delimiter=";",header=0,na_values="NaN")
data=data.dropna()
data


List of cols

In [29]:
data_cols=data.columns
data_cols

In [30]:
instance_name=data.iloc[0,1]
instance_name

'SQLMooveFleet'

Selecting cols of use

In [31]:
pertinent_cols=range(2,8)
pertinent_data_cols=data.iloc[:,pertinent_cols]
pertinent_data_cols

,BaseDeDatos,Esquema,TipoDeObjeto,NombreDeObjeto,Descripcion,Contenido
0,-,-,Instancia,SQLMooveFleet,Descripción de la instancia,Peso total: 837.01 GB - Número de bases de dat...
1,master,-,Base de Datos,master,Descripción de la Base de Datos,Tamaño en MB: 33.50
2,master,dbo,Esquema,dbo,Descripción de esquema,Objetos del esquema: 49
3,master,dbo,Procedimiento,sp_MScleanupmergepublisher,Descripción del procedimiento,create procedure dbo.sp_MScleanupmergepublishe...
4,master,dbo,Procedimiento,sp_MSrepl_startup,Descripción del procedimiento,create procedure dbo.sp_MSrepl_startup as ...
...,...,...,...,...,...,...
1453,tempdb,-,Base de Datos,tempdb,Descripción de la Base de Datos,"Tamaño en MB: 8,873.25"
1454,tempdb,dbo,Esquema,dbo,Descripción de esquema,Objetos del esquema: 49
1455,tempdb,guest,Esquema,guest,Descripción de esquema,Objetos del esquema: 0
1456,tempdb,INFORMATION_SCHEMA,Esquema,INFORMATION_SCHEMA,Descripción de esquema,Objetos del esquema: 0


Obtaining ddbb names

In [32]:
ddbb_names=data.BaseDeDatos.unique()
ddbb_names

array(['-', 'master', 'MooveCarsBI', 'msdb', 'tempdb'], dtype=object)

Selecting ddbb. Needs to adjust for each case. 

In [35]:
ddbb_selection=ddbb_names[[2,3]]
ddbb_selection

array(['MooveCarsBI', 'msdb'], dtype=object)

Selecting ddbb for documentation

In [36]:
pertinent_data=pertinent_data_cols[data['BaseDeDatos'].isin(ddbb_selection)]
pertinent_data

,BaseDeDatos,Esquema,TipoDeObjeto,NombreDeObjeto,Descripcion,Contenido
14,MooveCarsBI,-,Base de Datos,MooveCarsBI,Descripción de la Base de Datos,"Tamaño en MB: 847,570.69"
15,MooveCarsBI,anl,Esquema,anl,Descripción de esquema,Objetos del esquema: 134
16,MooveCarsBI,anl,Funcion,Split_On_Upper_Case,Descripción de la función,Create Function anl.Split_On_Upper_Case(@Temp ...
17,MooveCarsBI,anl,Procedimiento,actualiza_driver_by_tabla_corresp_conductordriver,Descripción del procedimiento,CREATE PROC [anl].[actualiza_driver_by_tabla...
18,MooveCarsBI,anl,Procedimiento,Actualizacion_tabla_conductor_driver,Descripción del procedimiento,create Procedure anl.Actualizacion_tabla_condu...
...,...,...,...,...,...,...
1448,msdb,SQLAgentOperatorRole,Esquema,SQLAgentOperatorRole,Descripción de esquema,Objetos del esquema: 0
1449,msdb,SQLAgentReaderRole,Esquema,SQLAgentReaderRole,Descripción de esquema,Objetos del esquema: 0
1450,msdb,SQLAgentUserRole,Esquema,SQLAgentUserRole,Descripción de esquema,Objetos del esquema: 0
1451,msdb,sys,Esquema,sys,Descripción de esquema,Objetos del esquema: 113


# Preparing to build a json 

Definition of some functions for separating text and the correct arrangement of of new files names are needed

Function for creating neccessary folder in route_base

In [38]:
def folders_creation(route_base):
    r"""
    Right now it only supports for paths of the type 'r"C:\Users\username\folder"'
    """
    if route_base[0]=='r':
        route_base=route_base[2:]
        route_base=route_base[:len(route_base)-1]

    folders_names=[ 'Vistas', 'Tablas', 'PAs', 'BBDD', 'Esquemas', 'Instancia' , 'Jobs']

    #loop for creating every folder
    for item in range(len(folders_names)):
        
        #select name of current folder and create path 
        thisfoldername=folders_names[item]
        folder_path=os.path.join(route_base,thisfoldername)
        
        #create a folder correctly named for storage in case it doesn't already exists
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

Function for separating cols names from content column

In [8]:
#needs a string with the contenido column of a table and returns a list with of all cols separatedly 
def split_cols_names(stringtext):
    
    #split string by cols names
    splitbycolsnames=re.split(",", stringtext)

    return splitbycolsnames


function for naming paths 

In [37]:
def file_path_namer(route_base =r"C:\Users\MiguelGarciaMoreno\Desktop\Jsons para doc", category='Table', instance='instance', ddbb='ddbb', schema='schema', table='table', folder_path=None ):
    r"""
    Generates a file path for storing JSON documents based on database instance, schema, and table names.
    
    Args:
        route_base (str, optional): The route to the folder where json files are stored. Defaults to "C:\\Users\\MiguelGarciaMoreno\\Desktop\\Jsons para doc".
        category (str, optional): The type of data treated. Can be 'Table', 'View', 'PA', 'Schema', 'ddbb', 'Jobs' or 'Instance', defaults to 'Table'.
        instance (str, optional): The database instance name. Defaults to 'instance'.
        ddbb (str, optional): The database name. Defaults to 'ddbb'.
        schema (str, optional): The schema name. Defaults to 'schema'.
        table (str, optional): The table name. Defaults to 'table'.
        folder_path (str, optional): The base folder path where the JSON file will be stored. 
                                     If not provided, defaults to a predefined location.
    
    Returns:
        str: The full file path where the JSON document should be saved.
    
    Example:
        ```python
        path = file_path_namer(instance="prod", ddbb="sales", schema="public", table="orders")
        print(path)
        ```
        Output:
        ```
        C:\Users\MiguelGarciaMoreno\Desktop\Jsons para doc\Tablas\prod_sales_tablas\prod_sales_public_tablas.json
        ```
    """  
    
    bar='_'

    # Raise a simple warning
    warnings.warn("At least folder_path or route_base argument must be provided, otherwise the saving will only work in the laptop of Miguel Garcia", category=UserWarning)

    if category == 'Table':
        if folder_path == None:
            folder_path=os.path.join(route_base,'Tablas',instance+bar+ddbb+bar+'tablas')
    
        #create a folder correctly named for storage in case it doesn't already exists
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
    
        #file_path for new json document
        file_path_string = os.path.join(folder_path, instance+bar+ddbb+bar+schema+bar+'tablas'+'.json')
        
    elif category == 'View': 
        if folder_path == None:
            folder_path=os.path.join(route_base,'Vistas',instance+bar+ddbb+bar+'vistas')
    
        #create a folder correctly named for storage in case it doesn't already exists
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
    
        #file_path for new json document
        file_path_string = os.path.join(folder_path, instance+bar+ddbb+bar+schema+bar+'vistas'+'.json')
        
    elif category == 'PA': 
        if folder_path == None:
            folder_path=os.path.join(route_base,'PAs',instance+bar+ddbb+bar+'pas')
    
        #create a folder correctly named for storage in case it doesn't already exists
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
    
        #file_path for new json document
        file_path_string = os.path.join(folder_path, instance+bar+ddbb+bar+schema+bar+'pas'+'.json')
        
    elif category == 'Schema':
        if folder_path == None:
            folder_path=os.path.join(route_base,'Esquemas',instance+bar+ddbb+bar+'esquemas')
    
        #create a folder correctly named for storage in case it doesn't already exists
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
    
        #file_path for new json document
        file_path_string = os.path.join(folder_path, instance+bar+ddbb+bar+schema+'.json')

    elif category == 'ddbb':
        if folder_path == None:
            folder_path=os.path.join(route_base,'BBDD')
    
        #create a folder correctly named for storage in case it doesn't already exists
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
    
        #file_path for new json document
        file_path_string = os.path.join(folder_path, instance+bar+ddbb+'.json')

    elif category == 'Jobs':
        if folder_path == None:
            folder_path=os.path.join(route_base,'Jobs')
    
        #create a folder correctly named for storage in case it doesn't already exists
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
    
        #file_path for new json document
        file_path_string = os.path.join(folder_path, instance+bar+'jobs'+'.json')
        
    elif category == 'Instance':
        if folder_path == None:
            folder_path=os.path.join(route_base,'Instancia')
    
        #create a folder correctly named for storage in case it doesn't already exists
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
    
        #file_path for new json document
        file_path_string = os.path.join(folder_path, instance+'.json')
         
    else:
        raise ValueError("Category type is not listed. Choose from: 'Table', 'View', 'PA', 'Schema', 'ddbb', 'Jobs' or 'Instance'")

    return file_path_string 

Function for where there is no data for tables, views or pas  

In [15]:
def na_data(f, missing_values='All'):
    """
    Generates a JSON-like structure representing missing data categories and writes it to a file.

    Args:
        missing_values (str, optional): Specifies the category of missing data. 
            Accepted values:
            - 'All' (default): Uses the schema name as the root.
            - 'Table': Uses "Tablas" as the root.
            - 'View': Uses "Vistas" as the root.
            - 'PA': Uses "PAs" as the root.
        f (file-like object): A file object where the output will be written.

    Writes:
        A JSON-like hierarchical structure with a "name" key and a "children" array 
        containing an object with `"name": "NA"`.

    Example output (when missing_values='Table'):
        {
            "name": "Tablas",
            "children": [
                { "name": "NA" }
            ]
        }
    """

    if missing_values == 'All':
        print( '{ "name": "NA"}',file=f)
    elif missing_values == 'Table':
        print( f'{{ "name": "Tablas",',file=f)
        print(' \t"children": [{ \n \t \t "name" : "NA" }] }',file=f) 
    elif missing_values == 'View':
        print( f'{{ "name": "Vistas",',file=f)
        print(' \t"children": [{ \n \t \t "name" : "NA" }] }',file=f)
    elif missing_values == 'PA':
        print( f'{{ "name": "PAs",',file=f)
        print(' \t"children": [{ \n \t \t "name" : "NA" }] }',file=f)

Function for splitting steps and commands (from jobs procdures)

In [16]:
def split_steps_and_commands(stringtext):

    #split string by steps and declare a new chain
    splitbysteps=re.split(r", P\d:", stringtext)
    splitbystepsandcommands=[]

    #after being stplitted by steps we split by commands instances also 
    for item in range(len(splitbysteps)):
        newtext=re.split("- COMMAND:",splitbysteps[item])
        splitbystepsandcommands.append(newtext[0])
        splitbystepsandcommands.append(newtext[1])

    #the result is a set containing steps names and commands alternatively
    #we now proceed to create a string for names and another one for commands
    leng=len(splitbystepsandcommands)
    steps=[]
    commands=[]

    for item in range(leng):
        if item%2==0:
            steps.append(splitbystepsandcommands[item])
        else:
            commands.append(splitbystepsandcommands[item])

    return steps, commands


Function for renaming commands (from jobs procdures) correctly

In [17]:
def rename_commands(commands):
    #in here we recover the COMMAND that was present before instructions
    for item in range(len(commands)):
        commands[item]='COMMAND:'+commands[item]
    return commands

Function for renaming steps (from jobs procdures) correctly

In [18]:
def rename_steps(steps):
    #in here we recover those P1, P2, etc headers for steps
    if len(steps) != 1:
        for item in range(len(steps)-1):
            steps[item+1]='P'+str(item+2)+':'+steps[item+1]
    return steps

Function for deciding whethet to keep steps descriptions in command sentences

In [19]:
def too_large_commands(string):
    #we decide wheter to include the command or set a 'Too large' string instead. 
    if len(string)>50:
        return "Demasiado largo"
    return string

# Functions of converters to json

Function for obtaining schemas in ddbb

In [20]:
def ddbb_schemas (ddbb):
    """
    Needs the string with the name of the database and returns a list with all the names of its schemas

    Example:
    ddbb_schemas(MooveCarsBI)
    array(['anl', 'bck', 'BI', 'dba', 'dbo', 'DM_BI', 'DWH', 'guest',
       'INFORMATION_SCHEMA', 'job', 'mov', 'stg', 'sys', 'uber_fi',
       'uber_xtr'], dtype=object)
    
    """
    array=pertinent_data[pertinent_data['BaseDeDatos']==ddbb].iloc[:,1].unique()[1:]

    return array

Function for creating the json file from cols names

Contenido to json. The content that harbours the cols names of each table 

In [10]:
def contenido_to_json(content_string, file):
    """
    Converts a list of column names into a JSON-like formatted structure and writes it to a file.

    This function processes a given list of column names (from `content_string`), formats them
    into a JSON-like structure, and writes them line by line into the specified file.

    Args:
        content_string (list): A list of column names to be converted into JSON format.
        file (file object): An open file object where the formatted JSON-like content is written.

    Behavior:
        - The first item in `content_string` is treated as the opening of the JSON object.
        - Intermediate items are formatted as JSON key-value pairs.
        - The last item is treated as the closing of the JSON structure.
        - If `content_string` has only one item, it is closed immediately.

    Example:
        Given `content_string = ["id", "name", "age"]`, the output written to `file` will be:

        ```json
        {
            "name": "id"
        },
        {
            "name": "name"
        },
        {
            "name": "age"
        }
        ```

    Note:
        The function does not return a value but writes directly to the file.

    """
    
    

    #splitting cols names
    splitted_cols_names=split_cols_names(content_string)

    #need for iterate
    leng = len(splitted_cols_names)

    
    for item in range(leng):
        col_name=splitted_cols_names[item]
        
        #distinct cases for we need to distguish the opening, the middle body and the final part where clauses are closed 
        if item == 0:
            print('\t \t { \n \t \t'+f' "name": "{col_name}"', file = file) 
            #special case of one instance 
            if leng == 1:
                print('}', file=file)
            else:
                print('\t \t},', file= file )
            
        elif item==range(leng)[leng-1]:
            print(f'\t \t{{  \n \t \t"name": "{col_name}" \n \t \t}}', file =file) 
        else:    
            print(f'\t \t{{ \n \t \t "name": "{col_name}" \n \t \t}},', file = file ) 

Function for creating tables' jsons 

In [55]:
def table_to_json(data, ddbb, schema, f):
    """
    Converts a table of data into a JSON format and writes it to a file.

    Args:
        data (pd.DataFrame): A pandas DataFrame containing the data to be converted.
        ddbb (str): The name of the database. (Not used in the current implementation, 
                    but could be relevant for future extensions).
        schema (str): The schema of the database. (Not used in the current implementation, 
                      but could be relevant for future extensions).
        f (file-like object): A file object where the resulting JSON data will be written.

    Writes:
        A JSON representation of the data is printed to the file-like object `f`.
        The JSON structure is hierarchical, with each item from `data` represented as 
        a "name" with a nested "children" array.

    The function distinguishes between the first item, middle items, and the last item 
    in the `data` to ensure proper JSON formatting (i.e., handling of commas and closing 
    brackets).

    Note:
        - The `newdata` DataFrame must have 'NombreDeObjeto' and 'Contenido' columns.
        - The function assumes that the JSON structure requires the conversion of each row 
          in `data` into a dictionary with nested children.
    
    output example:
        {
            "name": "Tablas",
            "children": [
                {
                 "name": "descripcion de la tabla",
                },
                {
                    "name": "table1",
                    "children": [...]
                },
                {
                    "name": "tabale2",
                    "children": [...]
                }
            ]
        }
    
    """

    #need for iteration
    leng=len(data)
    
    for item in range(leng):
            name = data.NombreDeObjeto.iloc[item]
            content = data.Contenido.iloc[item]
            table_description=data[(data['TipoDeObjeto']=='Tabla') & (data['NombreDeObjeto']==name)].iloc[0,4]
        
            #distinct cases for we need to distguish the opening, the middle body and the final part where clauses are closed 
            if item==0:
                print( '{ "name": "Tablas",',file=f)
                print(' \t "children": [ {'+f' "name": "{name}",',file=f) 
                print(f'\t \t "children" : [',file=f)
                print(f'\t\t\t {{ "name" : "{table_description}"}},', file=f)
                contenido_to_json(content,f)
                
                if leng==1:
                    print('\t ]',file=f)
                    print('}]}',file=f)
                else:
                    print('\t ]',file=f)
                    print('},',file=f)
                    
            elif item==range(leng)[leng-1]:
                print(f'\t {{ "name": "{name}",',file=f) 
                print(f'\t \t "children" : [',file=f)
                print(f'\t\t\t {{ "name" : "{table_description}"}},', file=f)
                contenido_to_json(content,f)
                print('] } ] }',file=f)
            else:    
                print(f'\t {{ "name": "{name}",',file=f) 
                print(f'\t \t "children" : [',file=f)
                print(f'\t\t\t {{ "name" : "{table_description}"}},', file=f)
                contenido_to_json(content,f)
                print('] },',file=f)

Function for creating view properties json file. This will include col_names and code

In [65]:
def views_properties_to_json(data,ddbb,schema,f,item):
    """
    Converts view properties from a dataset into a JSON format and writes them to a file.

    Parameters:
    data (DataFrame): A Pandas DataFrame containing view details.
    ddbb (str): The name of the database.
    schema (str): The schema to which the view belongs.
    f (file object): A writable file object where the JSON data will be written.
    item (int): The index of the row in the DataFrame to process.

    Functionality:
    - Extracts the 'Contenido' column from the DataFrame at the given index.
    - Splits the content into code and fields using '. Campos:' as a delimiter.
    - Writes the extracted code as a JSON object to the file.
    - Calls `contenido_to_json(fields, f)` to process and write the fields section.
    """

    content=data.Contenido.iloc[item]
 
    #split content in code and fields separatedly
    splitted_content=re.split('. Campos:',content)
    fields=splitted_content[1]
    code=splitted_content[0]

    print(f'{{ "name" : "{code}"}},',file=f)
    contenido_to_json(fields,f)








Function for creating view json files 

In [64]:
 def view_to_json(data,ddbb,schema,f):
    """
    Converts a table of data into a JSON format and writes it to a file.
    
    Args:
        data (pd.DataFrame): A pandas DataFrame containing the data to be converted.
        ddbb (str): The name of the database. (Not used in the current implementation, 
                    but could be relevant for future extensions).
        schema (str): The schema of the database. (Not used in the current implementation, 
                      but could be relevant for future extensions).
        f (file-like object): A file object where the resulting JSON data will be written.
    
    Writes:
        A JSON representation of the data is printed to the file-like object `f`.
        The JSON structure is hierarchical, with each item from `data` represented as 
        a "name" with a nested "children" array.
    
    The function distinguishes between the first item, middle items, and the last item 
    in the `data` to ensure proper JSON formatting (i.e., handling of commas and closing 
    brackets).
    
    Note:
        - The `newdata` DataFrame must have 'NombreDeObjeto' and 'Contenido' columns.
        - The function assumes that the JSON structure requires the conversion of each row 
          in `data` into a dictionary with nested children.
    
    Example of output:
    { "name": "Vistas",
 	"children": [{ 
 	 	 "name": "antiguedad_actual_conductores",
		"children" : [{ "name" : "Descripción de la vista"},
                     { "name" : "Codigo: CREATE view [anl].[antiguedad_actual_conductores] as... "},
            	 	 { 
             	 	 "name": " antiguedad"
            	 	},
            	 	{ 
             	 	 "name": "antiguedad_real" 
             	 	},
            	 	{  
             	 	"name": "jornadasdescontar" 
             	 	}
                    ]
	 	},
	 	{ 
 	 	"name": "ausencia_dia_2023",
		"children" : [{ "name" : "Descripción de la vista"},
            { "name" : "Codigo:           CREATE view [anl].[ausencia_dia_2023] as ...},
    	 	 { 
     	 	 "name": " amonestacion"
    	 	},
    	 	{ 
     	 	 "name": "Baja" 
     	 	},
    	 	{ 
     	 	 "name": "conductorid" 
     	 	}
            ]
        }]
    }
        
    """
     
    #need for iteration 
    leng = len(data)
     
    for item in range(leng):
        name = data.NombreDeObjeto.iloc[item]
        view_description=data.Descripcion.iloc[item]
        
        if item==0:
            print( '{ "name": "Vistas",',file=f)
            print(' \t"children": [{ \n \t \t'+f' "name": "{name}",',file=f)
            print(f'\t\t"children" : [{{ "name" : "{view_description}"}},',file=f)
            views_properties_to_json(data,ddbb,schema,f,item)
            print(']',file=f)
            
            #special case of one instance 
            if leng==1:
                print('\t \t}',file=f)
                print('\t]',file=f)
                print('}',file=f)
            else:
                 print('\t \t},',file=f)
            
            
        elif item==range(leng)[leng-1]:
            print(f'\t \t{{  \n \t \t"name": "{name}",',file=f)
            print(f'\t\t"children" : [{{ "name" : "{view_description}"}},',file=f)
            views_properties_to_json(data,ddbb,schema,f,item)
            print(']',file=f)
            print('\t \t}] \n \t} ',file=f)
            
        else:    
            print(f'\t \t{{ \n \t \t"name": "{name}",',file=f)
            print(f'\t\t"children" : [{{ "name" : "{view_description}"}},',file=f)
            views_properties_to_json(data,ddbb,schema,f,item)
            print(']',file=f)
            print('\t \t},',file=f) 
         

function for creating pas json files 

In [24]:
def PA_to_json(data,ddbb,schema,f):
    """
    Converts a table of data into a JSON format and writes it to a file.
    
    Args:
        data (pd.DataFrame): A pandas DataFrame containing the data to be converted.
        ddbb (str): The name of the database. (Not used in the current implementation, 
                    but could be relevant for future extensions).
        schema (str): The schema of the database. (Not used in the current implementation, 
                      but could be relevant for future extensions).
        f (file-like object): A file object where the resulting JSON data will be written.
    
    Writes:
        A JSON representation of the data is printed to the file-like object `f`.
        The JSON structure is hierarchical, with each item from `data` represented as 
        a "name" with a nested "children" array.
    
    The function distinguishes between the first item, middle items, and the last item 
    in the `data` to ensure proper JSON formatting (i.e., handling of commas and closing 
    brackets).
    
    Note:
        - The `newdata` DataFrame must have 'NombreDeObjeto' and 'Contenido' columns.
        - The function assumes that the JSON structure requires the conversion of each row 
          in `data` into a dictionary with nested children.
    """
     
    #need for iteration 
    leng = len(data)

    for item in range(leng):
        name = data.NombreDeObjeto.iloc[item]
        
        if item==0:
            print( '{ "name": "PAs",',file=f)
            print(' \t"children": [{ \n \t \t'+f' "name": "{name}"',file=f) 
        
            #special case of one instance 
            if leng==1:
                print('\t \t}',file=f)
                print('\t]',file=f)
                print('}',file=f)
            else:
                 print('\t \t},',file=f)
            
            
        elif item==range(leng)[leng-1]:
            print(f'\t \t{{  \n \t \t"name": "{name}" \n \t \t}}] \n \t}} ',file=f) 
        else:    
            print(f'\t \t{{ \n \t \t"name": "{name}" \n \t \t}},',file=f) 
        
     

function to create json files for schemas 

In [25]:
def schema_to_json(f, schema, table_data_exists, view_data_exists, pas_data_exists):
    """
    Generates a JSON-like hierarchical structure for a database schema and writes it to a file.

    Args:
        f (file-like object): A file object where the output JSON structure will be written.
        schema (str): The name of the schema being processed.
        table_data_exists (int): Flag indicating whether table data exists (1 if it exists, 0 otherwise).
        view_data_exists (int): Flag indicating whether view data exists (1 if it exists, 0 otherwise).
        pas_data_exists (int): Flag indicating whether procedural artifacts (PAs) exist (1 if they exist, 0 otherwise).

    Writes:
        A JSON structure representing the schema, including its tables, views, and procedural artifacts (PAs). 
        If no data exists for a category, a placeholder node with `"name": "NA"` is inserted.

    Process:
        - If no data exists in the schema, a `"NA"` node is added.
        - If tables exist, their JSON content is read from a corresponding file and added; otherwise, a `"NA"` node is used.
        - If views exist, their JSON content is read from a corresponding file and added; otherwise, a `"NA"` node is used.
        - If procedural artifacts (PAs) exist, their JSON content is read from a corresponding file and added; otherwise, a `"NA"` node is used.

    Example Output (when no data exists in the schema):
        {
            "name": "my_schema",
            "children": [
                { "name": "NA" }
            ]
        }

    Example Output (when only tables exist):
        {
            "name": "my_schema",
            "children": [
                { "name": "Tablas", "children": [...] },
                { "name": "Vistas", "children": [{ "name": "NA" }] },
                { "name": "PAs", "children": [{ "name": "NA" }] }
            ]
        }
    """

    print(f'{{ "name" : "{schema}",',file=f)
    print('"children": [ ',file=f)

    #if there is no info in that schema make it being floowed by NA 
    if table_data_exists+view_data_exists+pas_data_exists == 0:
        na_data(f)
    
    else: 
        #open table json file and paste it to the current schema json file. If there are no tables it will create a node branch with 'NA'
        if table_data_exists == 1:
            with open(file_path_namer(category='Table', instance=instance_name, ddbb=ddbb, schema=schema), 'r') as readdata:
               table_section=readdata.read()
            print(table_section,file=f)
        else: 
            na_data(f, missing_values='Table')
        print(',',file=f)
        
        #open views json file and paste it to the current schema json file. If there are no views it will create a node branch with 'NA'
        if view_data_exists == 1:
            with open(file_path_namer(category='View', instance=instance_name, ddbb=ddbb, schema=schema), 'r') as readdata:
               table_section=readdata.read()
            print(table_section,file=f)
        else: 
            na_data(f, missing_values='View')
        print(',',file=f)


        
        #open pas json file and paste it to the current schema json file. If there are no pas it will create a node branch with 'NA'
        if pas_data_exists == 1:
            with open(file_path_namer(category='PA', instance=instance_name, ddbb=ddbb, schema=schema), 'r') as readdata:
               table_section=readdata.read()
            print(table_section,file=f)
        else: 
            na_data(f, missing_values='PA')

    print(']}',file=f)

function for creating jsons files for ddbb

In [26]:
def ddbb_to_json(f, schemas, data):
    """
    Generates a JSON-like hierarchical structure representing a database (`ddbb`) and its schemas,
    then writes it to a file.

    Args:
        f (file-like object): A file object where the output JSON structure will be written.
        schemas (list of str): A list of schema names that belong to the database.
        data (dataframe): A data frame that contains all pertinent data, that means that id column has been left out.

    Writes:
        A JSON structure representing the database, with each schema included as a child node.
        If the database contains only one schema, it is directly closed. Otherwise, commas are
        inserted between schema objects for proper JSON formatting.

    Process:
        - The function iterates through the list of schemas.
        - For the first schema:
            - Writes the opening JSON structure for the database.
            - Reads and inserts the schema's JSON content.
            - If it's the only schema, closes the JSON structure immediately.
            - Otherwise, adds a comma for further schemas.
        - For intermediate schemas:
            - Reads and inserts the schema's JSON content.
            - Adds a comma for formatting.
        - For the last schema:
            - Reads and inserts the schema's JSON content.
            - Closes the JSON structure properly.

    Example Output (for a database with two schemas):
        {
            "name": "my_database",
            "children": [
                {
                 "name": "descripcion de la ddbb",
                },
                {
                    "name": "schema1",
                    "children": [...]
                },
                {
                    "name": "schema2",
                    "children": [...]
                }
            ]
        }
    """
    #need for iteration
    leng=len(schemas)

    #initiate loop
    for item in range(leng):
        current_schema=schemas[item]
        ddbb_description=data[(data['TipoDeObjeto']=='Base de Datos') & (data['BaseDeDatos']==ddbb)].iloc[0,4]
        
        if item==0:
            print( f'{{ "name": "{ddbb}",',file=f)
            print('\t"children": [',file=f) 
            print(f'\t\t {{ "name" : "{ddbb_description}" }},',file=f)

            #read current schema in loop and plug it
            with open(file_path_namer(category='Schema', instance=instance_name, ddbb=ddbb, schema=current_schema), 'r') as readdata:
               schema_section=readdata.read()
            print(schema_section,file=f)
        
            #special case of one instance 
            if leng==1:
                print('\t]',file=f)
                print('}',file=f)
            else:
                 print(',',file=f)
            
        elif item == range(leng)[leng-1]:
            #read current schema in loop and plug it
            with open(file_path_namer(category='Schema', instance=instance_name, ddbb=ddbb, schema=current_schema), 'r') as readdata:
               schema_section=readdata.read()
            print(schema_section,file=f)
            print('] }',file=f) 
        else:
            #read current schema in loop and plug it
            with open(file_path_namer(category='Schema', instance=instance_name, ddbb=ddbb, schema=current_schema), 'r') as readdata:
               schema_section=readdata.read()
            print(schema_section,file=f)
            print(',',file=f) 

Function for creating jobs json files

In [27]:
def jobs_to_json(f, jobdata):
    """
    Converts job data into a JSON-like hierarchical structure and writes it to a file.

    Args:
        f (file-like object): A file object where the output JSON structure will be written.
        jobdata (DataFrame): A pandas DataFrame containing job information with the following columns:
            - 'NombreDeObjeto' (str): The name of the job.
            - 'Contenido' (str): The text containing steps and commands.

    Writes:
        A JSON structure representing jobs, where each job contains a list of steps, 
        and each step contains a command. If a command is too long, it is processed 
        using `too_large_commands()`.

    Process:
        - Iterates through `jobdata`, extracting job names and corresponding step-command text.
        - Splits steps and commands using `split_steps_and_commands()`.
        - Renames steps and commands for better readability with `rename_steps()` and `rename_commands()`.
        - Constructs a hierarchical JSON structure where:
            - The root node is `"Jobs"`.
            - Each job is a child node containing its steps.
            - Each step is a child node containing its command.
        - Properly formats JSON with correct closing brackets and commas.

    Example Output (for two jobs with one step each):
        {
            "name": "Jobs",
            "children": [
                {
                    "name": "Job1",
                    "children": [
                        {
                            "name": "Step1",
                            "children": [
                                { "name": "Command1" }
                            ]
                        }
                    ]
                },
                {
                    "name": "Job2",
                    "children": [
                        {
                            "name": "StepA",
                            "children": [
                                { "name": "CommandA" }
                            ]
                        }
                    ]
                }
            ]
        }
    """

    #need for iteration
    leng=len(jobdata)
    
    for item in range(leng):
        #extraction of names and steps and command text
        jobname = jobdata.NombreDeObjeto.iloc[item]
        stepstext=jobdata.Contenido.iloc[item]
    
        #split the steps and commands text
        (steps, commands)=split_steps_and_commands(stepstext)
    
        #Structiring correctly the string for better output 
        steps=rename_steps(steps)
        commands=rename_commands(commands)
    
        #Initialise code for jobs
        if item==0:
            print('{',file=f)
            print('"name" : "Jobs",',file=f)
            print('"children" : [',file=f)
            
        #going through the job name    
        print('\t {',file=f)
        print(f'\t "name" : "{jobname}",',file=f)
        print('\t "children" : [',file=f)
    
        #loop for doing the code for all steps and their commands
        for itemsteps in range(len(steps)):
            print(' \t \t {',file=f)
            print(f' \t \t"name" : "{steps[itemsteps]}",',file=f)
            print(' \t \t"children" : [{',file=f)    
            print(f' \t \t \t "name" : "{too_large_commands(commands[itemsteps])}"',file=f)  
            print(' \t \t \t }]',file=f) 
    
            #ending clause if its the last step/command of this job
            if itemsteps == len(steps)-1:
                print('\t \t }]',file=f)
            else:
                print('\t \t },',file=f) 
                
        #ending clause if its the last job of the list
        if item  == leng-1:
            print('\t }',file=f)
            print('\t ]',file=f)
            print('}',file=f)
        else:
            print('\t },',file=f)
            

# Processment of data and creation of jsons files

Creation of neccessay folders in route_base

In [28]:
route_base=input(r""" Insert the route base for folder creation. If '\' are used then insert an r before the path string. For example r"C:\Users\username\folder" """ )

 Insert the route base for folder creation. If '\' are used then insert an r before the path string. For example r"C:\Users\username\folder"  dd


In [69]:
route_base

'r"C:\\Users\\MiguelGarciaMoreno\\Desktop\\fsafd"'

In [78]:
folders_creation(route_base)

Code for creating a json file for each ddbb. It also creates all table, views, pas and schemas json files. 

In [71]:
for ddbb in ddbb_selection:
    #list all schemas in the ddbb 
    schemas=ddbb_schemas(ddbb)

    for schema in schemas:
        #initialize flags 
        table_data_exists=0
        view_data_exists=0
        pas_data_exists=0
        
        #Select data in ddbb and schema and also only Tables from it
        newdata=pertinent_data[
                    (pertinent_data['BaseDeDatos'] == ddbb) & 
                    (pertinent_data['Esquema'] == schema) & 
                    (pertinent_data['TipoDeObjeto'] == 'Tabla')
                ]

        #Create json table file only if schema has got tables in it
        if len(newdata) != 0:
            table_data_exists=1
            #table json creator
            f = open(file_path_namer(instance=instance_name,ddbb=ddbb,schema=schema), "a")
            table_to_json(newdata,ddbb,schema,f)    
            f.close()
        
                
            
        #Create views json file only if schema has got tables in it
        newdata=pertinent_data[
                            (pertinent_data['BaseDeDatos'] == ddbb) & 
                            (pertinent_data['Esquema'] == schema) & 
                            (pertinent_data['TipoDeObjeto'] == 'Vista')
                ]
        #views json creator
        if len(newdata) != 0:
            view_data_exists=1
            #view json creator
            f = open(file_path_namer(category='View', instance=instance_name,ddbb=ddbb,schema=schema), "a")
            view_to_json(newdata,ddbb,schema,f)    
            f.close()

        #Create pas json file only if schema has got tables in it
        newdata=pertinent_data[
                            (pertinent_data['BaseDeDatos'] == ddbb) & 
                            (pertinent_data['Esquema'] == schema) & 
                            (pertinent_data['TipoDeObjeto'] == 'Procedimiento')
                ]
       #pas json creator
        if len(newdata) != 0:
            pas_data_exists=1
            #pas json creator
            f = open(file_path_namer(category='PA', instance=instance_name,ddbb=ddbb,schema=schema), "a")
            PA_to_json(newdata,ddbb,schema,f)    
            f.close()
        
        #create json file for schema 
        f = open(file_path_namer(category='Schema', instance=instance_name,ddbb=ddbb,schema=schema), "a")
        schema_to_json(f,schema,table_data_exists, view_data_exists,  pas_data_exists)
        f.close()

    #create json file for bbdd
    f = open(file_path_namer(category='ddbb', instance=instance_name,ddbb=ddbb), "a")
    ddbb_to_json(f, schemas, pertinent_data)
    f.close()

C:\Users\MiguelGarciaMoreno\AppData\Local\Temp\ipykernel_10572\2025000083.py:32: UserWarning: At least folder_path or route_base argument must be provided, otherwise the saving will only work in the laptop of Miguel Garcia
  warnings.warn("At least folder_path or route_base argument must be provided, otherwise the saving will only work in the laptop of Miguel Garcia", category=UserWarning)


Code for obtaining jobs' json files 

In [72]:
jobdata=data[data['TipoDeObjeto'] == 'Job'].iloc[:,[5,7]]

f = open(file_path_namer(category='Jobs', instance=instance_name,ddbb=ddbb), "a")
jobs_to_json(f, jobdata)
f.close()

C:\Users\MiguelGarciaMoreno\AppData\Local\Temp\ipykernel_10572\2025000083.py:32: UserWarning: At least folder_path or route_base argument must be provided, otherwise the saving will only work in the laptop of Miguel Garcia
  warnings.warn("At least folder_path or route_base argument must be provided, otherwise the saving will only work in the laptop of Miguel Garcia", category=UserWarning)


Code for obtaining instance json file

In [73]:
f = open(file_path_namer(category='Instance', instance=instance_name), "a")

leng = len(ddbb_selection)
#instance_description=data[(data['TipoDeObjeto']=='Instancia') & (data['BaseDeDatos']==instance_name)].iloc[0,4]
instance_description='descripcion de la instancia'


for item in range(leng):
    ddbb=ddbb_selection[item]
    if item == 0:
        print(f'{{ "name" : "{instance_name}",',file=f)
        print('"children" : [',file=f)
        print(f'\t\t {{ "name" : "{instance_description}" }},',file=f)
    
    #reading ddbb 
    with open(file_path_namer(category='ddbb', instance=instance_name, ddbb=ddbb), 'r') as readdata:
               ddbb_section=readdata.read()
    print(ddbb_section,file=f)
    print(',',file=f)

with open(file_path_namer(category='Jobs', instance=instance_name), 'r') as readdata:
               job_section=readdata.read()
print(job_section,file=f)

print(']}',file=f)
f.close()

C:\Users\MiguelGarciaMoreno\AppData\Local\Temp\ipykernel_10572\2025000083.py:32: UserWarning: At least folder_path or route_base argument must be provided, otherwise the saving will only work in the laptop of Miguel Garcia
  warnings.warn("At least folder_path or route_base argument must be provided, otherwise the saving will only work in the laptop of Miguel Garcia", category=UserWarning)


# Pruebas

In [109]:
contenido_to_json(5, content, ddbb_selection[0], 'anl')

{ 
 	 	 "name": "CCC Centro"
	 	},
	 	{ 
 	 	 "name": " Centro" 
 	 	},
	 	{ 
 	 	 "name": " Días Naturales" 
 	 	},
	 	{ 
 	 	 "name": " Empleado" 
 	 	},
	 	{  
 	 	"name": " Empresa" 
 	 	}


In [95]:
pertinent_data[
    (pertinent_data['BaseDeDatos'] == ddbb_selection[0]) & 
    (pertinent_data['Esquema'] == 'anl') & 
    (pertinent_data['TipoDeObjeto'] == 'Tabla')
]


,BaseDeDatos,Esquema,TipoDeObjeto,NombreDeObjeto,Contenido
56,MooveCarsBI,anl,Tabla,BajasA3,"CCC Centro, Centro, Días Naturales, Empleado, ..."
57,MooveCarsBI,anl,Tabla,BajasFieSS,"Causa fin IT, Causa Fin Pago Delegado, Causa I..."
58,MooveCarsBI,anl,Tabla,calendario_conductor,"ajuste_minutes, BI_id, BloqueLibranzaFinDeSema..."
59,MooveCarsBI,anl,Tabla,calendario_Conductor_futuro,"ajuste_minutes, BloqueLibranzaFinDeSemanaId, c..."
60,MooveCarsBI,anl,Tabla,calendario_vacaciones_futuro_conductor,"BloqueLibranzaFinDeSemanaId, conductorid, D_se..."
61,MooveCarsBI,anl,Tabla,cambiosVehiculo,"Anterior, BI_id, Fecha, Id, NombreActor, TipoA..."
62,MooveCarsBI,anl,Tabla,conteoVacaciones,"Apellido1, Apellido2, BASE, bloque, conductori..."
63,MooveCarsBI,anl,Tabla,cuotas_vacaciones_diaria_2023,"Base, Cuota, Dia"
64,MooveCarsBI,anl,Tabla,DimFecha,"anio, fecha, Id, mes, semana"
65,MooveCarsBI,anl,Tabla,dimFecha_2025,"BI_Id, Fecha"


In [ ]:
leng=5
for item in range(leng):
        #splitting cols names
        splitted_cols_names=split_cols_names(content)
        col_name=splitted_cols_names[item]

In [108]:
content=pruebadata.Contenido.iloc[0]
content

'CCC Centro, Centro, Días Naturales, Empleado, Empresa, Fecha Fin, Fecha Inicio, Imagen, NASS, Nº Documento, Parte, Persona, Subtipo, Tipo'

In [1]:
content_string='Codigo:   create view spt_values as  select name collate database_default as name,   number,   type collate database_default as type,   low, high, status  from sys.spt_values  . Campos: high,low,name,number,status,type'

In [4]:
splitted=re.split('. Campos:', content_string)
splitted

['Codigo:   create view spt_values as  select name collate database_default as name,   number,   type collate database_default as type,   low, high, status  from sys.spt_values  ',
 ' high,low,name,number,status,type']

In [ ]:
view_to_json(data,ddbb,schema,f)